In [1]:
# Import required modules
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
import time
import re
today = datetime.today().date().strftime("%d_%b")

In [2]:
def scrapeIndividualStaffLink(url):
    """Returns Indvidual staffs directory links."""
    
    staffLink = []
    driver = webdriver.Chrome(r"/home/faysal/Documents/softwares/chromedriver")
    driver.maximize_window()
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    time.sleep(4)
    
    # Scrapes indvidual staffs links
    driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
    time.sleep(4)
    for lnk in driver.find_elements_by_css_selector("div.link-item a"):
        staffLink.append(lnk.get_attribute("href"))
        
    # Remove empty links (empty strings)
    staffLink = list(filter(lambda x: x, staffLink))
    return staffLink

# Call the function to scrape indvidual staffs links
staffLink = scrapeIndividualStaffLink("https://www.cardiffmet.ac.uk/management/staff/Pages/default.aspx")


# Scrape individual staffs info
# Variables to scrape
name = []
designation = []
dept = []
email = []
phone = []

# Initialize driver outside foor loop, otherwise it will open for every for loop iteration.
driver = webdriver.Chrome(r"/home/faysal/Documents/softwares/chromedriver")
driver.maximize_window()
for url in staffLink:
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    time.sleep(4)
    
    # Name
    try:
        name.append(driver.find_element_by_css_selector("h1").text.strip())
    except:
        name.append("na")
        
    # Designation
    try:
        designation.append(driver.find_element_by_css_selector("h4").text.strip())
    except:
        designation.append("na")
        
    # Email
    try:
        email.append(driver.find_elements_by_css_selector("p")[1].text.split("\n")[2])
    except:
        email.append("na")
        
    # Phone
    try:
        phone.append(driver.find_elements_by_css_selector("p")[1].text.split("\n")[1])
    except:
        phone.append("na")

In [4]:
# Processing and cleaning
phoneCleaned = list(map(lambda x: "".join(re.findall(r"[0-9\+()]", x)), phone))
emailCleaned = list(map(lambda x: "".join(re.findall(r"[\w\.-]+@[\w\.-]+", x)), email))

# Now create a datframe
df = pd.DataFrame({
    "name":name,
    "designation":designation,
    "email":emailCleaned,
    "phone":phoneCleaned
})

# Let's see what we have
df.head(10)

,name,designation,email,phone
0,Professor David Brooksbank,Dean of Cardiff School of Management,dbrooksbank@cardiffmet.ac.uk,+44(0)2920416365
1,Dr Kelvin Hughes,Deputy Dean,kmhughes@cardiffmet.ac.uk,+44(0)2920416361
2,Sian Rees,Associate Dean (Enterprise),,
3,Stella Diamantidi,na,,
4,Melanie Tibbatts,na,mtibbatts@cardiffmet.ac.uk,+44(0)2920416415
5,Dr Ricky Li,Senior Lecturer in Banking and Finance,,
6,Dr Mukul Madahar,na,mmadahar@cardiffmet.ac.uk,+44(0)2920416307
7,Barbara Barnes,Lecturer / Year 2 Tutor BABMS,bbarnes@cardiffmet.ac.uk,+44(0)2920416420
8,Dr Malcolm James,Senior Lecturer in Accounting and Taxation,mdjames@cardiffmet.ac.uk,+44(0)2920416372
9,Dr Nicola Bolton,na,,


In [5]:
# Write to a csv file
df.to_csv(f"cardiffUniversityStaffs_{today}.csv", index=None)